In [1]:
using LowRankApprox
using LinearAlgebra
using Jacobi
using Test

In [2]:
function TrilinearMap(Coord_E, xhat, yhat, zhat)
    """
        local numbering of Hex
    
        5--------7           z
      / |      / |            |
    6 --|---- 8  |            |
    |   |     |  |            /----y
    |   |     |  |           /
    |   1 -------3         x
    | /       | /
    2 --------4
    """
    m = length(xhat)
    N1 = @. 0.125*(1-xhat)*(1-yhat)*(1-zhat)
    N2 = @. 0.125*(1+xhat)*(1-yhat)*(1-zhat)
    N3 = @. 0.125*(1-xhat)*(1+yhat)*(1-zhat)
    N4 = @. 0.125*(1+xhat)*(1+yhat)*(1-zhat)
    N5 = @. 0.125*(1-xhat)*(1-yhat)*(1+zhat)
    N6 = @. 0.125*(1+xhat)*(1-yhat)*(1+zhat)
    N7 = @. 0.125*(1-xhat)*(1+yhat)*(1+zhat)
    N8 = @. 0.125*(1+xhat)*(1+yhat)*(1+zhat)
    N = [N1 N2 N3 N4 N5 N6 N7 N8]
    X = N * Coord_E
    # X(3,m), 1st row x, 2nd row y, 3rd row z
    X = X'
    # derivatives of shape functions with respect to xhat
    dN1_dxhat = @. -0.125*(1-yhat)*(1-zhat)
    dN2_dxhat = -dN1_dxhat
    dN3_dxhat = @. -0.125*(1+yhat)*(1-zhat)
    dN4_dxhat = -dN3_dxhat
    dN5_dxhat = @. -0.125*(1-yhat)*(1+zhat)
    dN6_dxhat = -dN5_dxhat
    dN7_dxhat = @. -0.125*(1+yhat)*(1+zhat)
    dN8_dxhat = -dN7_dxhat

    # derivatives of shape functions with respect to yhat
    dN1_dyhat = @. -0.125*(1-xhat)*(1-zhat)
    dN2_dyhat = @. -0.125*(1+xhat)*(1-zhat)
    dN3_dyhat = -dN1_dyhat
    dN4_dyhat = -dN2_dyhat
    dN5_dyhat = @. -0.125*(1-xhat)*(1+zhat)
    dN6_dyhat = @. -0.125*(1+xhat)*(1+zhat)
    dN7_dyhat = -dN5_dyhat
    dN8_dyhat = -dN6_dyhat

    # derivatives of shape functions with respect to zhat
    dN1_dzhat = @. -0.125*(1-xhat)*(1-yhat)
    dN2_dzhat = @. -0.125*(1+xhat)*(1-yhat)
    dN3_dzhat = @. -0.125*(1-xhat)*(1+yhat)
    dN4_dzhat = @. -0.125*(1+xhat)*(1+yhat)
    dN5_dzhat = -dN1_dzhat
    dN6_dzhat = -dN2_dzhat
    dN7_dzhat = -dN3_dzhat
    dN8_dzhat = -dN4_dzhat
    
    # gradient of N, [dN/dxhat; dN/dyhat; dN/dzhat]
    GradN = zeros(3,m,8)
    GradN[1,:,:] = [dN1_dxhat dN2_dxhat dN3_dxhat dN4_dxhat dN5_dxhat dN6_dxhat dN7_dxhat dN8_dxhat]
    GradN[2,:,:] = [dN1_dyhat dN2_dyhat dN3_dyhat dN4_dyhat dN5_dyhat dN6_dyhat dN7_dyhat dN8_dyhat]
    GradN[3,:,:] = [dN1_dzhat dN2_dzhat dN3_dzhat dN4_dzhat dN5_dzhat dN6_dzhat dN7_dzhat dN8_dzhat]
    
    # JT = [[dx/dxhat, dy/dxhat, dz/dxhat],
    #       [dx/dyhat, dy/dyhat, dz/dyhat],
    #       [dx/dzhat, dy/dzhat, dz/dzhat]] (3m x 3)

    JTxhat = GradN[1,:,:] * Coord_E
    dxdxhat = JTxhat[:,1]
    dydxhat = JTxhat[:,2]
    dzdxhat = JTxhat[:,3]
    JTyhat = GradN[2,:,:] * Coord_E
    dxdyhat = JTyhat[:,1]
    dydyhat = JTyhat[:,2]
    dzdyhat = JTyhat[:,3]
    JTzhat = GradN[3,:,:] * Coord_E
    dxdzhat = JTzhat[:,1]
    dydzhat = JTzhat[:,2]
    dzdzhat = JTzhat[:,3]
    # compute det
    j1 = @. (dxdxhat*dydyhat*dzdzhat + dxdyhat*dydzhat*dzdzhat + dxdzhat*dydxhat*dzdyhat)
    j2 = @. (dxdxhat*dydzhat*dzdyhat + dxdyhat*dydxhat*dzdzhat + dxdzhat*dydyhat*dzdxhat)
    detJ = @. j1 - j2
    
    J = zeros(3,m,3)
    J[1,:,:] = [dxdxhat dxdyhat dxdzhat]
    J[2,:,:] = [dydxhat dydyhat dydzhat]
    J[3,:,:] = [dzdxhat dzdyhat dzdzhat]
    
    return X, J, detJ
end

TrilinearMap (generic function with 1 method)

In [3]:
function GetNormal(Coord_E, xhat, yhat, zhat, face)
    """
    Input:
    Coord_E: coordinate of physical element E as a 8x3 matrix
    xhat, yhat, zhat: are defined on the reference element Ehat
    and should be given as a vector (xhat=[1] or xhat =[1;1],...)
    face: that you want the normal
    Note the face and xhat, yhat, zhat should be consistent.
    
    Return:
    n: of size(3,m)
    le: length of n
    
    Based on following numbering:
    left:     nodes 1,2,5,6 at yhat = -1
    right:    nodes 3,4,7,8 at yhat =  1
    bottom:   nodes 1,2,3,4 at zhat = -1
    top:      nodes 5,6,7,8 at zhat =  1
    front:    nodes 2,4,6,8 at xhat =  1
    back:     nodes 1,3,5,7 at xhat = -1
    
    local numbering of Hex
    
        5--------7           z
      / |      / |            |
    6 --|---- 8  |            |
    |   |     |  |            /----y
    |   |     |  |           /
    |   1 -------3         x
    | /       | /
    2 --------4
    """
    X, J, detJ = TrilinearMap(Coord_E, xhat, yhat, zhat)

    dxdxhat = J[1,:,1]
    dxdyhat = J[1,:,2]
    dxdzhat = J[1,:,3]
    
    dydxhat = J[2,:,1]
    dydyhat = J[2,:,2]
    dydzhat = J[2,:,3]
    
    dzdxhat = J[3,:,1]
    dzdyhat = J[3,:,2]
    dzdzhat = J[3,:,3]

    m = length(xhat)
    
    if face == "left" && yhat == -ones(m)
        
        n1 = @. dydxhat*dzdzhat - dzdxhat*dydzhat
        n2 = @. dzdxhat*dxdzhat - dxdxhat*dzdzhat
        n3 = @. dxdxhat*dydzhat - dydxhat*dxdzhat
        leng = @. sqrt(n1*n1 + n2*n2 + n3*n3)
        n = zeros(3,m)
        n[1,:] = n1 ./ leng
        n[2,:] = n2 ./ leng
        n[3,:] = n3 ./ leng 
    
    elseif face == "right" && yhat == ones(m)
        
        n1 = @. dzdxhat*dydzhat - dydxhat*dzdzhat
        n2 = @. dxdxhat*dzdzhat - dzdxhat*dxdzhat
        n3 = @. dydxhat*dxdzhat - dxdxhat*dydzhat 
        leng = @. sqrt(n1*n1 + n2*n2 + n3*n3)
        n = zeros(3,m)
        n[1,:] = n1 ./ leng
        n[2,:] = n2 ./ leng
        n[3,:] = n3 ./ leng
        
    elseif face == "bottom" && zhat == -ones(m)
        
        n1 = @. dzdxhat*dydyhat - dydxhat*dzdyhat
        n2 = @. dxdxhat*dzdyhat - dzdxhat*dxdyhat
        n3 = @. dydxhat*dxdyhat - dxdxhat*dydyhat 
        leng = @. sqrt(n1*n1 + n2*n2 + n3*n3)
        n = zeros(3,m)
        n[1,:] = n1 ./ leng
        n[2,:] = n2 ./ leng
        n[3,:] = n3 ./ leng
        
    elseif face == "top" && zhat == ones(m)
        
        n1 = @. dydxhat*dzdyhat - dzdxhat*dydyhat 
        n2 = @. dzdxhat*dxdyhat - dxdxhat*dzdyhat 
        n3 = @. dxdxhat*dydyhat - dydxhat*dxdyhat 
        leng = @. sqrt(n1*n1 + n2*n2 + n3*n3)
        n = zeros(3,m)
        n[1,:] = n1 ./ leng
        n[2,:] = n2 ./ leng
        n[3,:] = n3 ./ leng
        
    elseif face == "front" && xhat == ones(m)
        
        n1 = @. dydyhat*dzdzhat - dzdyhat*dydzhat
        n2 = @. dzdyhat*dxdzhat - dxdyhat*dzdzhat
        n3 = @. dxdyhat*dydzhat - dydyhat*dxdzhat
        leng = @. sqrt(n1*n1 + n2*n2 + n3*n3)
        n = zeros(3,m)
        n[1,:] = n1 ./ leng
        n[2,:] = n2 ./ leng
        n[3,:] = n3 ./ leng
        
    elseif face == "back" && xhat == -ones(m)
        
        n1 = @. dzdyhat*dydzhat - dydyhat*dzdzhat 
        n2 = @. dxdyhat*dzdzhat - dzdyhat*dxdzhat
        n3 = @. dydyhat*dxdzhat - dxdyhat*dydzhat 
        leng = @. sqrt(n1*n1 + n2*n2 + n3*n3)
        n = zeros(3,m)
        n[1,:] = n1 ./ leng
        n[2,:] = n2 ./ leng
        n[3,:] = n3 ./ leng 
    else
        error("face is not defined")
    
    end

    return n, leng
end

GetNormal (generic function with 1 method)

# Enrichment Vectors
In BDDF 1987 paper, they considered 6 enrichments for cubic element which for linear case are given as
$$
\hat{S}_1 = \textbf{curl} \{
	\begin{bmatrix} 0 \\ 0 \\ \hat{x}\hat{y}\hat{z} \end{bmatrix},
    \begin{bmatrix} 0 \\ 0 \\ \hat{x}\hat{y}^2 \end{bmatrix},
	\begin{bmatrix} \hat{x}\hat{y}\hat{z} \\ 0 \\ 0 \end{bmatrix}, 
	\begin{bmatrix} \hat{y}\hat{z}^2 \\ 0 \\ 0 \end{bmatrix},
	\begin{bmatrix} 0 \\ \hat{x}\hat{y}\hat{z} \\ 0 \end{bmatrix},
    \begin{bmatrix} 0 \\ \hat{x}^2\hat{z} \\ 0 \end{bmatrix}
	\}
$$

In 2012, Wheeler, Xue and Yotov added another 6 enrichments to make the dimension of prime basis 24. They defined
$$
\hat{S}_2 = \textbf{curl} \{
	\begin{bmatrix} 0 \\ 0 \\ \hat{x}^2\hat{z} \end{bmatrix},
    \begin{bmatrix} 0 \\ 0 \\ \hat{x}^2\hat{y}\hat{z}\end{bmatrix},
	\begin{bmatrix} \hat{x}\hat{y}^2 \\ 0 \\ 0 \end{bmatrix}, 
	\begin{bmatrix} \hat{x}\hat{y}^2\hat{z} \\ 0 \\ 0 \end{bmatrix},
	\begin{bmatrix} 0 \\ \hat{y}\hat{z}^{2} \\ 0 \end{bmatrix},
    \begin{bmatrix} 0 \\ \hat{x}\hat{y}\hat{z}^2 \\ 0 \end{bmatrix}
	\}
$$

Following their idea, we will add 12 ernichment basis. We construct these extra basis using Orthogonal Polynomial in 2D at each face of the cubic element. We use Q, R = poly_qr2d(n1, n2, quad_face), which gives the combination of the following basis for the case of n1 = 3, n2 = 3 (number of 1D polynomial basis in direction 1 and 2)
$$
\{1, \hat{x}, \hat{x}^2, \hat{y}, \hat{y}^2, \hat{x}\hat{y}, \hat{x}^2\hat{y}, \hat{x}\hat{y}^2, \hat{x}^2\hat{y}^2 \}
$$

In [4]:
include("OrthoPoly2D.jl")

poly_QtQ (generic function with 1 method)

In [5]:
function GetEnrichment(xhat, yhat, zhat, face, n1, n2)
    """
    Based on following numbering:
    left:     nodes 1,2,5,6 at yhat = -1
    right:    nodes 3,4,7,8 at yhat =  1
    bottom:   nodes 1,2,3,4 at zhat = -1
    top:      nodes 5,6,7,8 at zhat =  1
    front:    nodes 2,4,6,8 at xhat =  1
    back:     nodes 1,3,5,7 at xhat = -1
    
    local numbering of Hex
    
        5--------7           z
      / |      / |            |
    6 --|---- 8  |            |
    |   |     |  |            /----y
    |   |     |  |           /
    |   1 -------3         x
    | /       | /
    2 --------4
    """
    
    Coord_Ehat = [-1. -1. -1.;1. -1. -1.;-1. 1. -1.;1. 1. -1.;-1. -1. 1.;1. -1. 1.;-1. 1. 1.;1. 1. 1.]
    left_face = [Coord_Ehat[1,1] Coord_Ehat[1,3];
                 Coord_Ehat[2,1] Coord_Ehat[2,3];
                 Coord_Ehat[5,1] Coord_Ehat[5,3];
                 Coord_Ehat[6,1] Coord_Ehat[6,3]]
    
    right_face = [Coord_Ehat[3,1] Coord_Ehat[3,3];
                  Coord_Ehat[4,1] Coord_Ehat[4,3];
                  Coord_Ehat[7,1] Coord_Ehat[7,3];
                  Coord_Ehat[8,1] Coord_Ehat[8,3]]
    
    bottom_face = [Coord_Ehat[1,1] Coord_Ehat[1,2];
                   Coord_Ehat[2,1] Coord_Ehat[2,2];
                   Coord_Ehat[3,1] Coord_Ehat[3,2];
                   Coord_Ehat[4,1] Coord_Ehat[4,2]]
    
    top_face = [Coord_Ehat[5,1] Coord_Ehat[5,2];
                Coord_Ehat[6,1] Coord_Ehat[6,2];
                Coord_Ehat[7,1] Coord_Ehat[7,2];
                Coord_Ehat[8,1] Coord_Ehat[8,2]]
    
    front_face = [Coord_Ehat[2,2] Coord_Ehat[2,3];
                  Coord_Ehat[4,2] Coord_Ehat[4,3];
                  Coord_Ehat[6,2] Coord_Ehat[6,3];
                  Coord_Ehat[8,2] Coord_Ehat[8,3]]
    
    back_face = [Coord_Ehat[1,2] Coord_Ehat[1,3];
                 Coord_Ehat[3,2] Coord_Ehat[3,3];
                 Coord_Ehat[5,2] Coord_Ehat[5,3];
                 Coord_Ehat[7,2] Coord_Ehat[7,3]]
    
    m = length(xhat)
    
    if face == "bottom"
        # in this case we have curl(0,0,f_i(x,y)), i=1:n1*n2, Q[i]=f_i(x,y)
        # curl(0,0,f_i(x,y)) = (df_i/dy, -df_i/dx, 0)
        Q, R = poly_qr2d(n1, n2, bottom_face)
        dQdx = []
        dQdy = []
        for i = 1:n1*n2
            push!(dQdx, polyderiv2d(Q[i],"d1"))
            push!(dQdy, polyderiv2d(Q[i],"d2"))
        end
        
        S = zeros(3,m,n1*n2)
        for i = 1:n1*n2
            S[1,:,i] = polyval2d(dQdy[i], xhat, yhat)
            S[2,:,i] = -polyval2d(dQdx[i], xhat, yhat)
            S[3,:,i] = zeros(m)
        end
        
    elseif face == "top"
        
        # in this case we have curl(0,0,f_i(x,y)*z), i=1:9, Q[i]=f_i(x,y)
        # curl(0,0,f_i(x,y)*z) = (df_i/dy*z, -df_i/dx*z, 0)
        Q, R = poly_qr2d(n1, n2, top_face)
        dQdx = []
        dQdy = []
        for i = 1:n1*n2
            push!(dQdx, polyderiv2d(Q[i],"d1"))
            push!(dQdy, polyderiv2d(Q[i],"d2"))
        end
        
        S = zeros(3,m,n1*n2)
        for i = 1:n1*n2
            S[1,:,i] = polyval2d(dQdy[i], xhat, yhat) .* zhat
            S[2,:,i] = -polyval2d(dQdx[i], xhat, yhat) .* zhat
            S[3,:,i] = zeros(m)
        end
        
    elseif face == "left"
        
        # in this case we have curl(0,f_i(x,z),0), i=1:9, Q[i]=f_i(x,z)
        # curl(0,f_i(x,z),0) = (-df_i/dz, 0, df_i/dx)
        Q, R = poly_qr2d(n1, n2, left_face)
        dQdx = []
        dQdz = []
        for i = 1:n1*n2
            push!(dQdx, polyderiv2d(Q[i],"d1"))
            push!(dQdz, polyderiv2d(Q[i],"d2"))
        end
        
        S = zeros(3,m,n1*n2)
        for i = 1:n1*n2
            S[1,:,i] = -polyval2d(dQdz[i], xhat, zhat)
            S[2,:,i] = zeros(m)
            S[3,:,i] = polyval2d(dQdx[i], xhat, zhat)
        end
        
    elseif face == "right"
        
        # in this case we have curl(0,f_i(x,z)*y,0), i=1:9, Q[i]=f_i(x,z)
        # curl(0,f_i(x,z)*y,0) = (-df_i/dz*y, 0, df_i/dx*y)
        Q, R = poly_qr2d(n1, n2, right_face)
        dQdx = []
        dQdz = []
        for i = 1:n1*n2
            push!(dQdx, polyderiv2d(Q[i],"d1"))
            push!(dQdz, polyderiv2d(Q[i],"d2"))
        end
        
        S = zeros(3,m,n1*n2)
        for i = 1:n1*n2
            S[1,:,i] = -polyval2d(dQdz[i], xhat, zhat) .* yhat
            S[2,:,i] = zeros(m)
            S[3,:,i] = polyval2d(dQdx[i], xhat, zhat) .* yhat
        end
        
    elseif face == "back"
        
        # in this case we have curl(f_i(y,z),0,0), i=1:9, Q[i]=f_i(y,z)
        # curl(f_i(y,z),0,0) = (0, df_i/dz, -df_i/dy)
        Q, R = poly_qr2d(n1, n2, back_face)
        dQdy = []
        dQdz = []
        for i = 1:n1*n2
            push!(dQdy, polyderiv2d(Q[i],"d1"))
            push!(dQdz, polyderiv2d(Q[i],"d2"))
        end
        
        S = zeros(3,m,n1*n2)
        for i = 1:n1*n2
            S[1,:,i] = zeros(m)
            S[2,:,i] = polyval2d(dQdz[i], yhat, zhat)
            S[3,:,i] = -polyval2d(dQdy[i], yhat, zhat)
        end
        
    elseif face == "front"
        
        # in this case we have curl(f_i(y,z),0,0), i=1:9, Q[i]=f_i(y,z)
        # curl(f_i(y,z),0,0) = (0, df_i/dz, -df_i/dy)
        Q, R = poly_qr2d(n1, n2, front_face)
        dQdy = []
        dQdz = []
        for i = 1:n1*n2
            push!(dQdy, polyderiv2d(Q[i],"d1"))
            push!(dQdz, polyderiv2d(Q[i],"d2"))
        end
        
        S = zeros(3,m,n1*n2)
        for i = 1:n1*n2
            S[1,:,i] = zeros(m)
            S[2,:,i] = polyval2d(dQdz[i], yhat, zhat) .* xhat
            S[3,:,i] = -polyval2d(dQdy[i], yhat, zhat) .* xhat
        end
        
    else
        error("face is not defined!")
        
    end
        
    return S
    
end

GetEnrichment (generic function with 1 method)

In [6]:
function PrimeBasis(Coord_E, xhat, yhat, zhat, n1, n2)

    m = length(xhat)

    # X are in E
    X, J, detJ = TrilinearMap(Coord_E, xhat, yhat, zhat)
    x = X[1,:]
    y = X[2,:]
    z = X[3,:]
    
    # the first 12 is defined on E using x,y,z
    P = zeros(3,m,12 + 6*n1*n2)
    P[1,:,1] = ones(m)
    P[1,:,2] = x
    P[1,:,3] = y
    P[1,:,4] = z
    P[2,:,5] = ones(m)
    P[2,:,6] = x
    P[2,:,7] = y
    P[2,:,8] = z
    P[3,:,9] = ones(m)
    P[3,:,10] = x
    P[3,:,11] = y
    P[3,:,12] = z
    
    S_1 = GetEnrichment(xhat, yhat, zhat, "left", n1, n2)
    S_2 = GetEnrichment(xhat, yhat, zhat, "right", n1, n2)
    S_3 = GetEnrichment(xhat, yhat, zhat, "top", n1, n2)
    S_4 = GetEnrichment(xhat, yhat, zhat, "bottom", n1, n2)
    S_5 = GetEnrichment(xhat, yhat, zhat, "front", n1, n2)
    S_6 = GetEnrichment(xhat, yhat, zhat, "back", n1, n2)
    # supplement 
    for i = 1:n1*n2
    P[1,:,12 + i] = (J[1,:,1] .* S_1[1,:,i] + J[1,:,2] .* S_1[2,:,i] + J[1,:,3] .* S_1[3,:,i]) ./ detJ
    P[2,:,12 + i] = (J[2,:,1] .* S_1[1,:,i] + J[2,:,2] .* S_1[2,:,i] + J[2,:,3] .* S_1[3,:,i]) ./ detJ
    P[3,:,12 + i] = (J[3,:,1] .* S_1[1,:,i] + J[3,:,2] .* S_1[2,:,i] + J[3,:,3] .* S_1[3,:,i]) ./ detJ
    
    P[1,:,12 + 1*n1*n2 + i] = (J[1,:,1] .* S_2[1,:,i] + J[1,:,2] .* S_2[2,:,i] + J[1,:,3] .* S_2[3,:,i]) ./ detJ
    P[2,:,12 + 1*n1*n2 + i] = (J[2,:,1] .* S_2[1,:,i] + J[2,:,2] .* S_2[2,:,i] + J[2,:,3] .* S_2[3,:,i]) ./ detJ
    P[3,:,12 + 1*n1*n2 + i] = (J[3,:,1] .* S_2[1,:,i] + J[3,:,2] .* S_2[2,:,i] + J[3,:,3] .* S_2[3,:,i]) ./ detJ
        
    P[1,:,12 + 2*n1*n2 + i] = (J[1,:,1] .* S_3[1,:,i] + J[1,:,2] .* S_3[2,:,i] + J[1,:,3] .* S_3[3,:,i]) ./ detJ
    P[2,:,12 + 2*n1*n2 + i] = (J[2,:,1] .* S_3[1,:,i] + J[2,:,2] .* S_3[2,:,i] + J[2,:,3] .* S_3[3,:,i]) ./ detJ
    P[3,:,12 + 2*n1*n2 + i] = (J[3,:,1] .* S_3[1,:,i] + J[3,:,2] .* S_3[2,:,i] + J[3,:,3] .* S_3[3,:,i]) ./ detJ
        
    P[1,:,12 + 3*n1*n2 + i] = (J[1,:,1] .* S_4[1,:,i] + J[1,:,2] .* S_4[2,:,i] + J[1,:,3] .* S_4[3,:,i]) ./ detJ
    P[2,:,12 + 3*n1*n2 + i] = (J[2,:,1] .* S_4[1,:,i] + J[2,:,2] .* S_4[2,:,i] + J[2,:,3] .* S_4[3,:,i]) ./ detJ
    P[3,:,12 + 3*n1*n2 + i] = (J[3,:,1] .* S_4[1,:,i] + J[3,:,2] .* S_4[2,:,i] + J[3,:,3] .* S_4[3,:,i]) ./ detJ
    
    P[1,:,12 + 4*n1*n2 + i] = (J[1,:,1] .* S_5[1,:,i] + J[1,:,2] .* S_5[2,:,i] + J[1,:,3] .* S_5[3,:,i]) ./ detJ
    P[2,:,12 + 4*n1*n2 + i] = (J[2,:,1] .* S_5[1,:,i] + J[2,:,2] .* S_5[2,:,i] + J[2,:,3] .* S_5[3,:,i]) ./ detJ
    P[3,:,12 + 4*n1*n2 + i] = (J[3,:,1] .* S_5[1,:,i] + J[3,:,2] .* S_5[2,:,i] + J[3,:,3] .* S_5[3,:,i]) ./ detJ
    
    P[1,:,12 + 5*n1*n2 + i] = (J[1,:,1] .* S_6[1,:,i] + J[1,:,2] .* S_6[2,:,i] + J[1,:,3] .* S_6[3,:,i]) ./ detJ
    P[2,:,12 + 5*n1*n2 + i] = (J[2,:,1] .* S_6[1,:,i] + J[2,:,2] .* S_6[2,:,i] + J[2,:,3] .* S_6[3,:,i]) ./ detJ
    P[3,:,12 + 5*n1*n2 + i] = (J[3,:,1] .* S_6[1,:,i] + J[3,:,2] .* S_6[2,:,i] + J[3,:,3] .* S_6[3,:,i]) ./ detJ
        
    end
    
    return P
end

PrimeBasis (generic function with 1 method)

In [7]:
function GetQuadrature2D(Q1d)
    """
    Input:
    Q: number of quadrature points in 1D over [-1,1]
    
    Return:Gauss Quadrature over [-1,1]^2
    qx:
    qy:
    w2:
    """
    # 1D Gauss
    q = zgj(Q1d, 0.0, 0.0)
    w = wgj(q, 0.0, 0.0)
    
    w2 = zeros(Q1d*Q1d)
    qx = zeros(Q1d*Q1d)
    qy = zeros(Q1d*Q1d)
    for i=1:Q1d
        for j=1:Q1d
            k = (i-1)*Q1d +j
            qx[k] = q[j]
            qy[k] = q[i]
            w2[k] = w[j]*w[i]
        end
    end
    return w2, qx, qy
end

function Legendre2D(x,y,i,j)
    # Legendre2D define on [-1,1]^2
    return legendre(x,i) .* legendre(y,j)
end

Legendre2D (generic function with 1 method)

In [8]:
function IntegrateFace(i,j,Coord_E, face, Q1d, n1, n2)
    """
    Input:
    i,j: degrees of Legendre polynomials L_i(s)L_j(t)
    coord_E: physical coordinate of element E
    face: face that you want to compute the normal component of the trace
    Q1d_quad: No. pts in 1D for integration over quad faces
    Q_tri: No. pts in 2D for integration over triangle faces
    
    Return:
    this function returns int{P.n * L_i(s)L_j(t)}dA
    where "n" is the normal on the physical face
    "L" are Legendre polynomial

    Based on following numbering:
    left:     nodes 1,2,5,6 at yhat = -1
    right:    nodes 3,4,7,8 at yhat =  1
    bottom:   nodes 1,2,3,4 at zhat = -1
    top:      nodes 5,6,7,8 at zhat =  1
    front:    nodes 2,4,6,8 at xhat =  1
    back:     nodes 1,3,5,7 at xhat = -1
    
    local numbering of Hex
    
        5--------7           z
      / |      / |            |
    6 --|---- 8  |            |
    |   |     |  |            /----y
    |   |     |  |           /
    |   1 -------3         x
    | /       | /
    2 --------4

    """
    # get quadrature on [-1,1]^2
    ww, q1, q2 = GetQuadrature2D(Q1d)
    m1 = -ones(Q1d*Q1d)
    p1 = ones(Q1d*Q1d)
    
    
    # left face yhat = -1
    if face == "left"
        n, le = GetNormal(Coord_E, q1, m1, q2, face)
        P = PrimeBasis(Coord_E, q1, m1, q2, n1, n2)
        PdotN = P[1,:,:] .* n[1,:] + P[2,:,:] .* n[2,:] + P[3,:,:] .* n[3,:]
        trace = PdotN' *(ww .* le .* Legendre2D.(q1, q2,i,j))
        
    # right face yhat = 1
    elseif face == "right"
        n, le = GetNormal(Coord_E, q1, p1, q2, face)
        P = PrimeBasis(Coord_E, q1, p1, q2, n1, n2)
        PdotN = P[1,:,:] .* n[1,:] + P[2,:,:] .* n[2,:] + P[3,:,:] .* n[3,:]
        trace = PdotN' *(ww .* le .* Legendre2D.(q1, q2,i,j))
        
    # back face at xhat = -1
    elseif face == "back"
        n, le = GetNormal(Coord_E, m1, q1, q2, face)
        P = PrimeBasis(Coord_E, m1, q1, q2, n1, n2)
        PdotN = P[1,:,:] .* n[1,:] + P[2,:,:] .* n[2,:] + P[3,:,:] .* n[3,:]
        trace = PdotN' *(ww .* le .* Legendre2D.(q1, q2,i,j))
    
    # front face at xhat = 1
    elseif face == "front"
        n, le = GetNormal(Coord_E, p1, q1, q2, face)
        P = PrimeBasis(Coord_E, p1, q1, q2, n1, n2)
        PdotN = P[1,:,:] .* n[1,:] + P[2,:,:] .* n[2,:] + P[3,:,:] .* n[3,:]
        trace = PdotN' *(ww .* le .* Legendre2D.(q1, q2,i,j))
        
    # top face at zhat = 1
    elseif face == "top"
        n, le = GetNormal(Coord_E, q1, q2, p1, face)
        P = PrimeBasis(Coord_E, q1, q2, p1, n1, n2);
        PdotN = P[1,:,:] .* n[1,:] + P[2,:,:] .* n[2,:] + P[3,:,:] .* n[3,:]
        trace = PdotN' *(ww .* le .* Legendre2D.(q1, q2,i,j))
        
    # bottom face includes nodes 1,2,3 at zhat = -1
    elseif face == "bottom"
        n, le = GetNormal(Coord_E, q1, q2, m1, face)
        P = PrimeBasis(Coord_E, q1, q2, m1, n1, n2);
        PdotN = P[1,:,:] .* n[1,:] + P[2,:,:] .* n[2,:] + P[3,:,:] .* n[3,:]
        trace = PdotN' *(ww .* le .* Legendre2D.(q1, q2,i,j))
    else
        error("face is not defined")
    end
    
    return trace
    
end

IntegrateFace (generic function with 1 method)

In [9]:
function ConstraintMat(Coord_E, Q1d, n1, n2)
    """
    Input:
    coord_E: physical coordinate of element E
    Q1d_quad: No. pts in 1D for integration over quad faces
    Q_tri: No. pts in 2D for integration over triangle faces
    
    Return:
    Constraint Matrix of size (27,22)
    """
    
    # On each quad face we compute trace_i,j
    # for (i,j) = (2,0),(2,1),(2,2),(1,2),(0,2) ==> 30 rows for all quad
    n = [2 2 2 1 0]
    quad = ["left", "right", "bottom", "top", "front", "back"]
    # 30 constraint
    L = zeros(30,12 + 6*n1*n2)
    for i = 1:6 # loop over faces
        for j = 1:length(n) # loop over legendre_i,j
            trace = IntegrateFace(n[j], n[length(n)+1-j], Coord_E, quad[i], Q1d, n1, n2)
            L[j + length(n)*(i-1),:] = trace
        end
    end
    
    return L
end

ConstraintMat (generic function with 1 method)

In [10]:
#Coord_E = [0. 0. 0.;1. 0. 0.;0. 1. 0.;1. 1. 0.;0. 0. 1.;1. 0. 1.;0. 1. 1.;1. 1. 1.]
Coord_E = [-1. -1. -1.;1. -1. -1.;-1. 1. -1.;1. 1. -1.;-1. -1. 1.;1. -1. 1.;-1. 1. 1.;1. 1. 1.]
Q1d = 5
n1 = 3
n2 = 3
L = ConstraintMat(Coord_E, Q1d, n1, n2)

30×66 Matrix{Float64}:
  0.0           0.0           0.0          …   2.77556e-17   9.71445e-17
  0.0           0.0           0.0              1.02696e-15  -1.09635e-15
  0.0           0.0           0.0              6.93889e-18  -1.38778e-17
  0.0           0.0           0.0              0.0           0.0
  0.0           0.0           0.0              2.77556e-17   0.0
  0.0           0.0           0.0          …   2.77556e-17  -9.71445e-17
  0.0           0.0           0.0              1.02696e-15   1.09635e-15
  0.0           0.0           0.0              6.93889e-18   1.38778e-17
  0.0           0.0           0.0              0.0           0.0
  0.0           0.0           0.0              2.77556e-17   0.0
  0.0           0.0           0.0          …   9.54098e-16   4.16334e-17
  0.0           0.0           0.0              4.85723e-17   1.19349e-15
  0.0           0.0           0.0             -5.20417e-17  -1.38778e-17
  ⋮                                        ⋱                

In [38]:
F=pqrfact(L',atol=1e-16)
F[:R]

24×30 UpperTrapezoidal{Float64}:
 -1.94678e-15   2.80953e-16   3.25865e-16  …  -2.01777e-17  -2.33185e-17
  0.0          -1.88734e-15   3.65072e-16     -2.96231e-17  -2.58949e-17
  0.0           0.0          -1.89435e-15      1.46616e-16  -4.82637e-17
  0.0           0.0           0.0             -1.10448e-17   1.36177e-16
  0.0           0.0           0.0              1.30633e-17   3.47216e-18
  0.0           0.0           0.0          …   3.90836e-18   1.81015e-18
  0.0           0.0           0.0              3.28835e-18   1.7228e-18
  0.0           0.0           0.0             -1.1359e-18   -1.92291e-19
  0.0           0.0           0.0              5.49004e-19  -1.78671e-18
  0.0           0.0           0.0              7.44104e-18   6.29057e-18
  0.0           0.0           0.0          …   8.63583e-18   3.15792e-18
  0.0           0.0           0.0              2.45926e-18  -4.06354e-18
  0.0           0.0           0.0              1.57762e-18   2.82745e-19
  0.0           0.0

In [40]:
Qfull, _ = qr(F[:Q])
VL = Qfull[:, 43:end]

66×24 Matrix{Float64}:
 -0.0488347    -0.146074    -0.0359714    …   0.0987206     0.0858863
 -0.0407052    -0.32334     -0.181085         0.226013      0.045349
  0.0105945     0.294297     0.566988         0.320497      0.085555
 -0.247167      0.415023    -0.182813         0.205874     -0.105153
 -0.0230985    -0.0422795   -0.0130713       -0.0295868     0.0587121
  0.0371666     0.110466    -0.00787118   …  -0.0543546     0.687045
  0.0267496     0.187724    -0.11394          0.0210467     0.184571
  0.137039      0.288584    -0.134717        -0.328054     -0.131917
  0.0164216     0.0626779    0.0391683        0.121639     -0.190929
  0.00607382   -0.0475703    0.00333623       0.0737026     0.0174024
 -0.0430712    -0.137943     0.144838     …  -0.415062     -0.115961
  0.0209629     0.160186     0.271029        -0.20997      -0.280572
  0.125595     -0.0738216   -0.529989        -0.00839981   -0.186654
  ⋮                                       ⋱                
  0.00474006   -0

In [41]:
function clip(x; threshold=1e-12)
    y = copy(x)
    y[abs.(y) .< threshold] .= 0
    y
end


clip (generic function with 1 method)

In [55]:
function GetNull(L, Mode)
    """
    Input:
    L: the constraint matrix created by ConstraintMat function
    mode: QR, or SVD
    
    Return:
    The 18 columns of V. (L = USV^T) which span null(L)
    """
    if Mode == "QR"
        F=pqrfact(L')
        Qfull, _ = qr(F[:Q])
        VL = Qfull[:, 43:end]
        
    elseif Mode == "SVD"
        
        U, S, V = svd(L,full=true)
        VL = V[:,43:end]
    else
        error("mode should be QR or SVD")
    end
    return VL
end

GetNull (generic function with 1 method)

In [70]:
function VondermondeMat(Coord_E, VL, n1 ,n2)
    """
    local numbering of Hex
    
        5--------7           z
      / |      / |            |
    6 --|---- 8  |            |
    |   |     |  |            /----y
    |   |     |  |           /
    |   1 -------3         x
    | /       | /
    2 --------4

    """
    nl, le = GetNormal(Coord_E, [0], [-1.],[0],"left")
    nr, le = GetNormal(Coord_E, [0], [1.],[0],"right")
    nbt, le = GetNormal(Coord_E, [0.], [0.],[-1.],"bottom")
    nt, le = GetNormal(Coord_E, [0.], [0.],[1.],"top")
    nf, le = GetNormal(Coord_E, [1.], [0.],[0.],"front")
    nbk, le = GetNormal(Coord_E, [-1.], [0.],[0.],"back")
    normals = [nbt nbt nbt nbt nl nl nl nl nf nf nf nf nr nr nr nr nbk nbk nbk nbk nt nt nt nt]
    nd1 = [-1.;-1.;-1.]
    nd2 = [1.;-1.;-1.]
    nd3 = [-1.;1.;-1.]
    nd4 = [1.;1.;-1.]
    nd5 = [-1.;-1.;1.]
    nd6 = [1.;-1.;1.]
    nd7 = [-1.;1.;1.]
    nd8 = [1.;1.;1.]
    nodes = [nd1 nd2 nd3 nd4 nd1 nd2 nd5 nd6 nd2 nd4 nd6 nd8 nd3 nd4 nd7 nd8 nd1 nd3 nd5 nd7 nd5 nd6 nd7 nd8]
    # vondermonde matrix, V_ij = phi_j(x_i).n_i
    VM = zeros(24,24)
    for i=1:24
        for j=1:24
            P = PrimeBasis(Coord_E, [nodes[1,i]], [nodes[2,i]], [nodes[3,i]], n1, n2)
            # reduced prime basis P, to dim=24 by multiplying VL from the right
            PP = zeros(3,1,24)
            PP[1,:,:] = P[1,:,:] * VL
            PP[2,:,:] = P[2,:,:] * VL
            PP[3,:,:] = P[3,:,:] * VL
            VM[i,j] = PP[1,1,j] * normals[1,i] + PP[2,1,j] * normals[2,i] + PP[3,1,j] * normals[3,i]
        end
    end

    return VM
end

VondermondeMat (generic function with 1 method)

In [71]:
function GetNodalBasis(Coord_E, xhat, yhat, zhat, Q1d, Mode, n1, n2)

    """
    This function returns Nodal basis as a (3,m,24) array, m = length(xhat)
    Input:
    Coord_E: coordinate of physical element E as a 8x3 matrix
    xhat,yhat,zhat (vector of size m): defined on reference element Ehat
    
    Return:
    Nhat of size(3,m,24): Nodal basis for Hex
    The order of these 24 nodal basis are:
    1-4:   dofs on bottom face,   nodes 1,2,3,4
    5-8:   dofs on left face,     nodes 1,2,5,6
    9-12:  dofs on front face,    nodes 2,4,6,8
    13-16: dofs on right face,    nodes 3,4,7,8
    17-20: dofs on back face,     nodes 1,3,5,7
    21-24: dofs on top face,      nodes 5,6,7,8
    
    local numbering of Hex
    
        5--------7           z
      / |      / |            |
    6 --|---- 8  |            |
    |   |     |  |            /----y
    |   |     |  |           /
    |   1 -------3         x
    | /       | /
    2 --------4
    
    """
    L = ConstraintMat(Coord_E, Q1d, n1, n2)
    VL = GetNull(L, Mode)
    # create nodal basis from vondermonde (VM) and VL
    P = PrimeBasis(Coord_E, xhat, yhat, zhat, n1, n2)
    VM = VondermondeMat(Coord_E, VL, n1, n2)
    invVM = inv(VM)
    # nodal basis
    m = length(xhat)
    Nhat = zeros(3,m,24)
    Nhat[1,:,:] = P[1,:,:] * VL * invVM
    Nhat[2,:,:] = P[2,:,:] * VL * invVM
    Nhat[3,:,:] = P[3,:,:] * VL * invVM
    
    return Nhat
end

GetNodalBasis (generic function with 1 method)

In [75]:
@testset "TestNodalBasisUniform" begin

    Coord_E = [-1. -1. -1.;1. -1. -1.;-1. 1. -1.;1. 1. -1.;-1. -1. 1.;1. -1. 1.;-1. 1. 1.;1. 1. 1.]

    nl, le = GetNormal(Coord_E, [0], [-1.],[0],"left")
    nr, le = GetNormal(Coord_E, [0], [1.],[0],"right")
    nbt, le = GetNormal(Coord_E, [0.], [0.],[-1.],"bottom")
    nt, le = GetNormal(Coord_E, [0.], [0.],[1.],"top")
    nf, le = GetNormal(Coord_E, [1.], [0.],[0.],"front")
    nbk, le = GetNormal(Coord_E, [-1.], [0.],[0.],"back")
    
    err = 1e-14
    Q1d = 5
    Mode = "QR"
    n1 = 3
    n2 = 3
    #============= Node 1==============#
    # check node 1
    Nhat = GetNodalBasis(Coord_E, [-1], [-1], [-1], Q1d, Mode, n1, n2)
    @test isapprox(dot(Nhat[:,1,1],nbt), 1.;atol=err)
    @test isapprox(dot(Nhat[:,1,5],nl), 1.;atol=err)
    @test isapprox(dot(Nhat[:,1,17],nbk), 1.;atol=err)
    
end

TestNodalBasisUniform: Error During Test at In[75]:1
  Got exception outside of a @test
  SingularException(7)
  Stacktrace:
    [1] checknonsingular
      @ /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.6/LinearAlgebra/src/factorization.jl:19 [inlined]
    [2] checknonsingular
      @ /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.6/LinearAlgebra/src/factorization.jl:21 [inlined]
    [3] #lu!#132
      @ /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.6/LinearAlgebra/src/lu.jl:85 [inlined]
    [4] #lu#136
      @ /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.6/LinearAlgebra/src/lu.jl:273 [inlined]
    [5] lu (repeats 2 times)
      @ /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.6/LinearAlgebra/src/lu.jl:272 [inlined]
    [6] inv(A::Matrix{Float64})
      @ LinearAlgebra /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.6/LinearAlgebra/src/dense.jl:815
    [7] G

LoadError: [91mSome tests did not pass: 0 passed, 0 failed, 1 errored, 0 broken.[39m